### Imports

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

try:
  from google.colab import drive
  drive.mount('/content/drive')

  COLAB = True
except ModuleNotFoundError:
  COLAB = False

# Data Gathering

TODO:

- [x] Create the sections (Bruno)
- [x] Create a structure to download from drive and create the pandas dataframe (Bruno)
- [x] Create a structure to load data in colab or locally in the same way, saving in a variable called `df` (Bruno)
- [ ] new idea

Download [here](https://drive.google.com/file/d/1hWST2LvCc4ai15ZtUMoC9vj0cDDf3izS/view?usp=drive_link) the CSV. Upload it to your own google-drive inside a folder called `predictive-analysis`

In [ ]:
if COLAB:
  df = pd.read_csv('/content/drive/MyDrive/predictive-analysis/essays.csv')
else:
  df = pd.read_csv("essays.csv")

df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate


# Data Cleaning

TODO:

- [x] Check the shape and the types of each column (Bruno)
- [x] Check the names of the categories in `discourse_text` and `discourse_effectiveness`, validate if they match with kaggle's specification. (Bruno)
- [ ] Check for outliers, especially in the column `discourse_text`

In [ ]:
df.shape

(36765, 5)

In [ ]:
df.describe()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
count,36765,36765,36765,36765,36765
unique,36765,4191,36691,7,3
top,c8a973681feb,900A879708F0,Summer projects should be student-designed,Evidence,Adequate
freq,1,23,14,12105,20977


In [ ]:
# it appears as an object because everything is a string. It is all good!
df.dtypes

discourse_id               object
essay_id                   object
discourse_text             object
discourse_type             object
discourse_effectiveness    object
dtype: object

In [ ]:
df.discourse_type.unique()

array(['Lead', 'Position', 'Claim', 'Evidence', 'Counterclaim',
       'Rebuttal', 'Concluding Statement'], dtype=object)

In [ ]:
df.discourse_effectiveness.unique()

array(['Adequate', 'Ineffective', 'Effective'], dtype=object)

# EDA

TODO:

- [x] show the distribution of each category in an ugly and lazy way (Bruno)
- [ ] replace the ugly and lazy presentation for nice and awesome graphs
- [ ] bring ideas to what kind of EDA we can do in the text column (`discourse_text`)


In [ ]:
df[["discourse_type"]].value_counts()

discourse_type      
Evidence                12105
Claim                   11977
Position                 4024
Concluding Statement     3351
Lead                     2291
Counterclaim             1773
Rebuttal                 1244
Name: count, dtype: int64

In [ ]:
df[["discourse_effectiveness"]].value_counts()

discourse_effectiveness
Adequate                   20977
Effective                   9326
Ineffective                 6462
Name: count, dtype: int64

In [ ]:
df[["discourse_type", "discourse_effectiveness"]].value_counts().sort_index()

discourse_type        discourse_effectiveness
Claim                 Adequate                   7097
                      Effective                  3405
                      Ineffective                1475
Concluding Statement  Adequate                   1945
                      Effective                   825
                      Ineffective                 581
Counterclaim          Adequate                   1150
                      Effective                   418
                      Ineffective                 205
Evidence              Adequate                   6064
                      Effective                  2885
                      Ineffective                3156
Lead                  Adequate                   1244
                      Effective                   683
                      Ineffective                 364
Position              Adequate                   2784
                      Effective                   770
                      Ineffective   